In [263]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [264]:
# Replace the path with the correct path for your data.
df = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [265]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [266]:
categorical_var = df.describe(include='O').columns
categorical_var = df[categorical_var]
categorical_var.nunique()

id                           421097
term                              2
int_rate                        110
grade                             7
sub_grade                        35
                              ...  
last_pymnt_d                     25
next_pymnt_d                      4
last_credit_pull_d               26
application_type                  2
verification_status_joint         3
Length: 26, dtype: int64

In [267]:
# converting id and interest rate to numeric
df['id'] = pd.to_numeric(df['id'], errors='coerce')
df['int_rate'] = pd.to_numeric(df['int_rate'].str.strip('%'))

In [268]:
# dropping categorical rows that have too many or too few unique values
for col in categorical_var.columns:
    if df[col].nunique() > 30 or df[col].nunique() == 1:
        df.drop(col, 1, inplace=True)

In [269]:
# dropping the last two rows that don't contain data

df = df[:-2]

In [270]:
from scipy.stats import chi2_contingency

# A function to find cramers value for two categorical variables
def cramers_v(x,y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r,k = confusion_matrix.shape
    return (chi2/(n*min((k-1),(r-1))))**.5

In [271]:
# Let's find the categorical variables most associated with loan status
categorical_cols = df.describe(include='O').columns
categorical_var = df[categorical_cols]

cramerv_values = {}

for col in categorical_var.columns:
    x = df[col]
    y = df['loan_status']
    cramerv_values[col] = cramers_v(x, y)
cramerv_values = sorted(cramerv_values.items(), key = lambda x: x[1])
cramerv_values

[('application_type', 0.007726412574411751),
 ('emp_length', 0.011614449191061946),
 ('purpose', 0.03160615087089259),
 ('title', 0.031940728119615465),
 ('home_ownership', 0.03642382729851921),
 ('next_pymnt_d', 0.03843496997279434),
 ('verification_status', 0.054478979202925194),
 ('issue_d', 0.0669618207904894),
 ('initial_list_status', 0.0766660904150539),
 ('grade', 0.09059541903269518),
 ('term', 0.09200651502010207),
 ('verification_status_joint', 0.11001013034043629),
 ('last_credit_pull_d', 0.3138093468675081),
 ('last_pymnt_d', 0.46443163453241276),
 ('loan_status', 1.0)]

The categorical feature most associated with loan_status is 'last_pymnt_d'.
##### I will add this feature to our model

In [272]:
categorical_features = pd.get_dummies(df[['last_pymnt_d']], drop_first=True)

##### Now let's look at the continuous variables

In [273]:
# First let's look at the correlation of all the continuous variables with our target variable
continous_cols = df.describe().columns
continuous_var = df[continous_cols]

# I converted the target to dummy variables to show the correlation
pd.options.display.max_rows = 100
print(pd.concat([continuous_var, pd.get_dummies(df['loan_status'])], axis=1).corr().iloc[:, -6:])
pd.options.display.max_rows = 10

                                 Current   Default  Fully Paid  \
member_id                       0.146156  0.004429   -0.119457   
loan_amnt                       0.015382  0.001988   -0.033211   
funded_amnt                     0.015382  0.001988   -0.033211   
funded_amnt_inv                 0.015401  0.001965   -0.033178   
installment                    -0.014418  0.002959   -0.007999   
annual_inc                      0.003351 -0.001643    0.017387   
dti                             0.004559  0.003209   -0.028381   
delinq_2yrs                     0.000854  0.000028   -0.013399   
inq_last_6mths                 -0.083236  0.009081    0.034420   
mths_since_last_delinq         -0.018144 -0.000026    0.028909   
mths_since_last_record         -0.044888  0.002225    0.031625   
open_acc                       -0.004811  0.003726   -0.006400   
pub_rec                        -0.013627  0.003274    0.004848   
revol_bal                       0.035122 -0.004585   -0.021251   
total_acc 

In [274]:
# The possible features are the features most correlated with our target variable
possible_features = ['collection_recovery_fee', 'recoveries', 'out_prncp_inv', 'total_rec_prncp', 'total_pymnt_inv',
                     'last_pymnt_amnt', 'out_prncp', 'total_pymnt']


In [275]:
# Let's check the intercorrelation between our possible features

df[possible_features].corr()

,collection_recovery_fee,recoveries,out_prncp_inv,total_rec_prncp,total_pymnt_inv,last_pymnt_amnt,out_prncp,total_pymnt
collection_recovery_fee,1.000000,0.973343,-0.128250,-0.100695,-0.039150,-0.045996,-0.128244,-0.039152
recoveries,0.973343,1.000000,-0.127730,-0.100933,-0.038240,-0.045958,-0.127724,-0.038244
out_prncp_inv,-0.128250,-0.127730,1.000000,-0.135268,0.048055,-0.371058,0.999997,0.048092
total_rec_prncp,-0.100695,-0.100933,-0.135268,1.000000,0.964892,0.817310,-0.135324,0.964876
total_pymnt_inv,-0.039150,-0.038240,0.048055,0.964892,1.000000,0.742462,0.048012,0.999997
last_pymnt_amnt,-0.045996,-0.045958,-0.371058,0.817310,0.742462,1.000000,-0.371040,0.742504
out_prncp,-0.128244,-0.127724,0.999997,-0.135324,0.048012,-0.371040,1.000000,0.048053
total_pymnt,-0.039152,-0.038244,0.048092,0.964876,0.999997,0.742504,0.048053,1.000000


In [276]:
# Because of the intercorrelation, I will only use two of the possible features as continuous features. 
continuous_features = ['last_pymnt_amnt', 'out_prncp']
continuous_features = df[continuous_features]

In [277]:
model_features = pd.concat([continuous_features, categorical_features], axis=1)
target = df['loan_status']

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(n_estimators=5, max_depth=3)
X = model_features
Y = target
X = X.dropna(axis=1)

print(cross_val_score(rfc, X, Y, cv=10))

[0.90643775 0.93935076 0.94395764 0.8920947  0.89679411 0.89145096
 0.88491771 0.90441473 0.89543306 0.90352919]


### Let' see what happens if we drop the categorical features from our model

In [278]:
model_features = continuous_features
target = df['loan_status']

rfc = ensemble.RandomForestClassifier(n_estimators=5, max_depth=3)
X = model_features
Y = target
X = X.dropna(axis=1)

print(cross_val_score(rfc, X, Y, cv=10))

[0.94433758 0.94547743 0.94533495 0.9441951  0.94673474 0.94668725
 0.94414496 0.94661474 0.94475978 0.94216976]


The model improved, and so it seems the categorical features hurt our model.

### Let's see if we can achieve above 90% accuracy without any features related to payment amount or outstanding principal.

In [279]:
# First let's see how our model performs with all the continous variables
model_features = continuous_var
target = df['loan_status'] 

rfc = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5)
X = model_features
Y = target
X = X.dropna(axis=1)

print(cross_val_score(rfc, X, Y, cv=10))

[0.95340885 0.95585476 0.95649593 0.9555698  0.94286393 0.95884588
 0.95314541 0.95620889 0.95183699 0.94936589]


At the moment the model performs better than 90%. Let's see if it stays this way after we drop the related features.

In [280]:
# First let's find out what is related to payment amount and outstanding principal
print('Correlation of out_prncp:')
print('')
print(df.corr()['out_prncp'].sort_values(ascending=False).head(10))
print('-------------------------------')
print('Correlation of last_pymnt_amnt:')
print('')
print(df.corr()['last_pymnt_amnt'].sort_values(ascending=False).head(10))

Correlation of out_prncp:

out_prncp           1.000000
out_prncp_inv       0.999997
total_rec_int       0.684606
loan_amnt           0.623926
funded_amnt         0.623926
funded_amnt_inv     0.623858
installment         0.506732
max_bal_bc          0.296159
annual_inc_joint    0.294799
total_bc_limit      0.231633
Name: out_prncp, dtype: float64
-------------------------------
Correlation of last_pymnt_amnt:

last_pymnt_amnt    1.000000
total_rec_prncp    0.817310
total_pymnt        0.742504
total_pymnt_inv    0.742462
installment        0.264130
funded_amnt_inv    0.259878
loan_amnt          0.259864
funded_amnt        0.259864
tot_hi_cred_lim    0.135725
tot_cur_bal        0.131646
Name: last_pymnt_amnt, dtype: float64


In [281]:
# Let's drop all the related features from our model

model_features = continuous_var.drop(['out_prncp', 'out_prncp_inv', 'total_rec_prncp',
                                              'total_pymnt', 'total_pymnt_inv'], axis=1)
target = df['loan_status'] 

rfc = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5)
X = model_features
Y = target
X = X.dropna(axis=1)

print(cross_val_score(rfc, X, Y, cv=10))

[0.82907079 0.87986512 0.89273586 0.77402579 0.88546664 0.86946094
 0.76385096 0.8720701  0.90372147 0.90526291]


After dropping all the related features, the model does not perform above 90%. There are no other features to add to the data set to get this number above 90%. Perhaps changing some of parameters of random forest might help. However, I still don't believe it is possible to get this model above 90% with those features removed.